In [ ]:
# 13/11/2020
# Using Keras Application Layers

In [ ]:
# importing all necessary libraries and functions

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# mounting drive 

from google.colab import drive
drive.mount('/content/drive')

# obtaining dataset

image_size = (256, 256)
batch_size = 32
epochs = 100

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/My Drive/Colab Notebooks/x_ray_tb/data/train_small" ,
    validation_split = 0.2 ,
    subset = "training",
    seed = 1337,
    image_size = image_size ,
    batch_size = batch_size ,   
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/My Drive/Colab Notebooks/x_ray_tb/data/train_small" ,
    validation_split = 0.2 ,
    subset = "validation",
    seed = 1337,
    image_size = image_size ,
    batch_size = batch_size , 
    
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 738 files belonging to 2 classes.
Using 591 files for training.
Found 738 files belonging to 2 classes.
Using 147 files for validation.


In [ ]:
# prefetching data

train_ds = train_ds.prefetch(buffer_size = 32)
val_ds = val_ds.prefetch(buffer_size = 32)

In [ ]:
def make_model(input_shape, pre_model):
    
    inputs = keras.Input(shape = input_shape)
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)

    # block one

    x = pre_model(include_top=False, weights='imagenet', input_shape=input_shape)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units=1, activation = "sigmoid")(x)
    return keras.Model(inputs, outputs)

In [ ]:
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import InceptionV3

def call():
  MODELS = [
            DenseNet169,
            InceptionResNetV2,
            InceptionV3,
            ]
  i = 1
  for mdls in MODELS:
    print()
    
    model = make_model(input_shape = image_size + (3,), pre_model = mdls)
    keras.utils.plot_model(model, show_shapes = True)

    print('##### '+str(i)+ ' #####')
    
    callbacks = [
                 keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                   factor=0.2, 
                                                   patience=5, 
                                                   verbose=1),
                 keras.callbacks.EarlyStopping(monitor='val_loss', 
                                               min_delta=0.01, 
                                               patience=5,
                                               verbose=1,
                                               mode='min')
    ]
    
    model.compile(
        optimizer = keras.optimizers.Adam(1e-3),
        loss = "binary_crossentropy",
        metrics = ["accuracy"],
        )
    
    model.fit(
        train_ds, 
        epochs=epochs, 
        callbacks=callbacks, 
        validation_data=val_ds
        )
    i=i+1

In [ ]:
call()


##### 1 #####
Epoch 1/100
19/19 [==============================] - 18s 927ms/step - loss: 15.3008 - accuracy: 0.5939 - val_loss: 2306963.2500 - val_accuracy: 0.4558
Epoch 2/100
19/19 [==============================] - 11s 583ms/step - loss: 3.5226 - accuracy: 0.6734 - val_loss: 142267.0469 - val_accuracy: 0.4558
Epoch 3/100
19/19 [==============================] - 11s 585ms/step - loss: 2.5384 - accuracy: 0.7056 - val_loss: 55098.5195 - val_accuracy: 0.4558
Epoch 4/100
19/19 [==============================] - 11s 580ms/step - loss: 1.0705 - accuracy: 0.7428 - val_loss: 33697.7500 - val_accuracy: 0.4558
Epoch 5/100
19/19 [==============================] - 11s 578ms/step - loss: 1.0314 - accuracy: 0.7648 - val_loss: 4750.1963 - val_accuracy: 0.4558
Epoch 6/100
19/19 [==============================] - 11s 576ms/step - loss: 0.7394 - accuracy: 0.7530 - val_loss: 20.4544 - val_accuracy: 0.4694
Epoch 7/100
19/19 [==============================] - 11s 581ms/step - loss: 0.7654 - accuracy: 0.